In [3]:
import math

import pandas as pd
import numpy as np
import sklearn
from jupyter_core.migrate import regex
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,StratifiedKFold
from sklearn.preprocessing import *
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,FunctionTransformer
from sklearn.metrics import *
from sklearn.compose import *
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.pipeline import  Pipeline
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA
from warnings import filterwarnings
from sklearn import set_config
filterwarnings('ignore')
set_config(display='diagram')

## Data Loading

In [4]:
df=pd.read_csv(r"C:\Users\dai\Downloads\assigment data\train.csv",index_col=0)
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Item_Identifier,,,,,,,,,,,
FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
df['Item_Type'].unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [6]:
df=df.drop(['Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

## Data Info

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8523 entries, FDA15 to DRG01
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Weight           7060 non-null   float64
 1   Item_Fat_Content      8523 non-null   object 
 2   Item_Visibility       8523 non-null   float64
 3   Item_Type             8523 non-null   object 
 4   Item_MRP              8523 non-null   float64
 5   Outlet_Size           6113 non-null   object 
 6   Outlet_Location_Type  8523 non-null   object 
 7   Outlet_Type           8523 non-null   object 
 8   Item_Outlet_Sales     8523 non-null   float64
dtypes: float64(4), object(5)
memory usage: 665.9+ KB


In [8]:
df.isnull().sum()

Item_Weight             1463
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
Item_Outlet_Sales          0
dtype: int64

## Cleaning and filtering

In [9]:
df.Item_Fat_Content.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [10]:
df['Item_Fat_Content']=df['Item_Fat_Content'].replace({'LF':"low fat",
                                                       "Low Fat": 'low fat',
                                                       "reg":"regular",
                                                       "Regular":'regular',
                                                       },regex=True)

In [11]:
df.Item_Fat_Content.unique()

array(['low fat', 'regular'], dtype=object)

## handling Null value

In [12]:
group_mean=df.groupby(['Item_Type','Item_Fat_Content'])['Item_Weight'].transform(lambda x : x.mean())
df['Item_Weight']=df['Item_Weight'].fillna(group_mean)

In [13]:
for key,value in df.groupby(['Outlet_Location_Type','Outlet_Type'])['Outlet_Size'].groups.items():
    print(key,df.loc[value]['Outlet_Size'].mode())

('Tier 1', 'Grocery Store') 0    Small
Name: Outlet_Size, dtype: object
('Tier 1', 'Supermarket Type1') 0    Medium
Name: Outlet_Size, dtype: object
('Tier 2', 'Supermarket Type1') 0    Medium
Name: Outlet_Size, dtype: object
('Tier 3', 'Grocery Store') 0    Medium
Name: Outlet_Size, dtype: object
('Tier 3', 'Supermarket Type1') 0    Medium
Name: Outlet_Size, dtype: object
('Tier 3', 'Supermarket Type2') 0    Medium
Name: Outlet_Size, dtype: object
('Tier 3', 'Supermarket Type3') 0    Medium
Name: Outlet_Size, dtype: object


In [14]:
df['Outlet_Size'].mode()

0    Medium
Name: Outlet_Size, dtype: object

In [15]:
imputer=SimpleImputer(strategy='most_frequent').set_output(transform='pandas')
df['Outlet_Size']=imputer.fit_transform(df[['Outlet_Size']])
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Item_Identifier,,,,,,,,,
FDA15,9.30,low fat,0.016047,Dairy,249.8092,Medium,Tier 1,Supermarket Type1,3735.1380
DRC01,5.92,regular,0.019278,Soft Drinks,48.2692,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.50,low fat,0.016760,Meat,141.6180,Medium,Tier 1,Supermarket Type1,2097.2700
FDX07,19.20,regular,0.000000,Fruits and Vegetables,182.0950,Medium,Tier 3,Grocery Store,732.3800
NCD19,8.93,low fat,0.000000,Household,53.8614,High,Tier 3,Supermarket Type1,994.7052


In [16]:
df.isna().sum()

Item_Weight             0
Item_Fat_Content        0
Item_Visibility         0
Item_Type               0
Item_MRP                0
Outlet_Size             0
Outlet_Location_Type    0
Outlet_Type             0
Item_Outlet_Sales       0
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8523 entries, FDA15 to DRG01
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Weight           8523 non-null   float64
 1   Item_Fat_Content      8523 non-null   object 
 2   Item_Visibility       8523 non-null   float64
 3   Item_Type             8523 non-null   object 
 4   Item_MRP              8523 non-null   float64
 5   Outlet_Size           8523 non-null   object 
 6   Outlet_Location_Type  8523 non-null   object 
 7   Outlet_Type           8523 non-null   object 
 8   Item_Outlet_Sales     8523 non-null   float64
dtypes: float64(4), object(5)
memory usage: 923.9+ KB


## Transformations

In [18]:
X,y=df.drop('Item_Outlet_Sales',axis=1),df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((6818, 8), (1705, 8), (6818,), (1705,))

#### features segregation

In [20]:
X['Item_Fat_Content'].value_counts()

Item_Fat_Content
low fat    5517
regular    3006
Name: count, dtype: int64

In [21]:
numerical_col=['Item_Weight','Item_Visibility','Item_MRP',]
categorical_col=['Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type']
ordinal_col=['Outlet_Size','Outlet_Location_Type','Item_Fat_Content']
item_fat_order=['low fat','regular']
outlet_location_order=['Tier 3','Tier 2',"Tier 1"]
outlet_size_order=['Small','Medium','High']
ordinal_order=[outlet_size_order,outlet_location_order,item_fat_order]
nominal_col=['Item_Type','Outlet_Type']

## Simple NUll values handling using Function

In [22]:
    def group_mean_imputer(X):
        impute_col = 'Item_Weight'
        group_cols = ['Item_Type', 'Item_Fat_Content']
        group_means = X.groupby(group_cols)[impute_col].transform('mean')
        X_imputed = X.copy()
        X_imputed[impute_col] = X[impute_col].fillna(group_means)
        return X_imputed[[impute_col] + group_cols]




In [23]:
trns1=ColumnTransformer(transformers=[
    ('item_weight_imputation',FunctionTransformer(func=group_mean_imputer,validate=False),['Item_Weight', 'Item_Type', 'Item_Fat_Content']),
    ('outlet_size_imputation',SimpleImputer(strategy='most_frequent'),['Outlet_Size'])
],remainder='passthrough',verbose_feature_names_out=False).set_output(transform='pandas')

trns2=ColumnTransformer(transformers=[
    ('scaler',StandardScaler(),numerical_col),
    ('Ohe',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),nominal_col),
    ("OE",OrdinalEncoder(categories=ordinal_order),ordinal_col)
],remainder='passthrough',verbose_feature_names_out=False).set_output(transform='pandas')

trns2.fit_transform(trns1.fit_transform(X))

,Item_Weight,Item_Visibility,Item_MRP,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,...,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Outlet_Size,Outlet_Location_Type,Item_Fat_Content
Item_Identifier,,,,,,,,,,,,,,,,,,,,,
FDA15,-0.840272,-0.970732,1.747454,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0
DRC01,-1.637899,-0.908111,-1.489023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
FDN15,1.094801,-0.956917,0.010040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0
FDX07,1.495975,-1.281758,0.660050,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
NCD19,-0.927586,-1.281758,-1.399220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FDF22,-1.414894,-0.181193,1.180783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0
FDS36,-1.057378,-0.371154,-0.527301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
NCJ29,-0.533492,-0.599784,-0.897208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


## Pipeline

In [24]:
pipe = Pipeline(steps=[
    ('preprocessing1',trns1),
    ('preprocessing2', trns2),
    # ('PCA',PCA(n_components=10)),
    # Placeholder for the actual model
    ('regressor', None)
])

In [25]:
pipe

,steps,"[('preprocessing1', ...), ('preprocessing2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('item_weight_imputation', ...), ('outlet_size_imputation', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


## Grid Search CV

In [26]:
param_grid_list = [
    # LinearRegression
    {
        'regressor': [LinearRegression()]
        # No hyperparameters to tune
    },

    # ElasticNet
    {
        'regressor': [ElasticNet()],
        'regressor__alpha':[0.01,0.1,1.0],
        'regressor__l1_ratio':[0.01,0.1,1.0],
        'regressor__fit_intercept':[True,False],
        'regressor__max_iter':[10000]
    },

    # SVR
    {
        'regressor': [SVR()],
        'regressor__kernel':['rbf','sigmoid'], # Reduced for speed
        'regressor__gamma':['auto',"scale"],
        "regressor__C":[0.1, 1.0]
    },

    # DecisionTreeRegressor (FIXED)
    {
        'regressor': [DecisionTreeRegressor(random_state=42)],
        "regressor__max_depth":[None, 5, 10],
        "regressor__min_samples_split":[2, 5],
        "regressor__min_samples_leaf":[1, 2, 4] # Added a commonly used parameter
    },

    # AdaBoostRegressor (using default estimator)
    {
        'regressor': [AdaBoostRegressor(random_state=42)],
        "regressor__n_estimators":[50, 100],
        'regressor__learning_rate':[0.1, 1.0],
    },

    # RandomForestRegressor
    {
        'regressor': [RandomForestRegressor(random_state=42)],
        "regressor__n_estimators":[100, 200],
        "regressor__max_depth":[None, 10],
    },

    # GradientBoostingRegressor
    {
        'regressor': [GradientBoostingRegressor(random_state=42)],
        "regressor__n_estimators":[100, 200],
        "regressor__learning_rate":[0.1, 1.0],
    },

    # XGBRegressor
    {
        'regressor': [XGBRegressor(random_state=42, eval_metric='rmse')],
        "regressor__learning_rate":[0.1, 0.5],
        "regressor__n_estimators":[100, 200]
    },

    # LGBMRegressor
    {
        'regressor':[LGBMRegressor(random_state=42)],
        'regressor__n_estimators':[100,200,300,500],
        'regressor__learning_rate':[0.01,0.1,1,1.2,3,5]
    }
]

## Best Model Selection

## R2score

In [27]:
import math
results_list = []
scoring_metric = 'r2'
best_estimators=None
for model_params in param_grid_list:
    model_name = model_params['regressor'][0]
    grid_search = GridSearchCV(estimator=pipe,param_grid=model_params,cv=5, scoring=scoring_metric,verbose=0,n_jobs=-1)
    grid_search.fit(X, y)
    results_list.append({
        'Model': model_name,
        # GridSearchCV reports negative score for loss metrics, so we negate it for clarity (MSE)
        'Best Parameters': grid_search.best_params_,
        'Best Score (Mean CV R2 Score)': grid_search.best_score_
    })


results_df = pd.DataFrame(results_list)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000875 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 785
[LightGBM] [Info] Number of data points in the train set: 8523, number of used features: 26
[LightGBM] [Info] Start training from score 2181.288915


In [28]:
results_df=results_df.reset_index().drop('index',axis=1)
results_df = results_df.sort_values(by='Best Score (Mean CV R2 Score)', ascending=False)
results_df

,Model,Best Parameters,Best Score (Mean CV R2 Score)
6,GradientBoostingRegressor(random_state=42),{'regressor': GradientBoostingRegressor(random...,0.593587
3,DecisionTreeRegressor(random_state=42),{'regressor': DecisionTreeRegressor(random_sta...,0.589308
8,LGBMRegressor(random_state=42),"{'regressor': LGBMRegressor(random_state=42), ...",0.589043
5,RandomForestRegressor(random_state=42),{'regressor': RandomForestRegressor(random_sta...,0.585563
4,AdaBoostRegressor(random_state=42),{'regressor': AdaBoostRegressor(random_state=4...,0.570403
7,"XGBRegressor(base_score=None, booster=None, ca...","{'regressor': XGBRegressor(base_score=None, bo...",0.570090
1,ElasticNet(),"{'regressor': ElasticNet(), 'regressor__alpha'...",0.559313
0,LinearRegression(),{'regressor': LinearRegression()},0.558212
2,SVR(),"{'regressor': SVR(), 'regressor__C': 1.0, 'reg...",0.167577


In [29]:
results_df.iloc[0]['Best Parameters']

{'regressor': GradientBoostingRegressor(random_state=42),
 'regressor__learning_rate': 0.1,
 'regressor__n_estimators': 100}

In [30]:
model=GradientBoostingRegressor(random_state=42,learning_rate=0.1,n_estimators=100)
model.fit(pipe.fit_transform(X),y)

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


## Test

In [37]:
test=pd.read_csv(r"C:\Users\dai\Downloads\assigment data\test.csv")
test.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [39]:
test_cleaned=test.drop(['Item_Identifier','Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

In [40]:
test_cleaned['Item_Fat_Content']=test_cleaned['Item_Fat_Content'].replace({'LF':"low fat",
                                                       "Low Fat": 'low fat',
                                                       "reg":"regular",
                                                       "Regular":'regular',
                                                       },regex=True)

In [42]:
test_trns=pipe.transform(test_cleaned)
y_pred=model.predict(test_trns)
test['Item_Outlet_Sales']=y_pred

In [64]:
test_df=test[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]
test_df['Item_Outlet_Sales']=test_df['Item_Outlet_Sales'].abs()
test_df=test_df.set_index(['Item_Identifier','Outlet_Identifier'])
test_df.to_csv('submission1.csv')

,,Item_Outlet_Sales
Item_Identifier,Outlet_Identifier,
